In [1]:
# %%
import pandas as pd
import pickle
import numpy as np 
import geopandas as gpd
from glob import glob
%cd ..
#from loveda_utils import map_label2rgb, idx2rgb, idx2label
from munich_utils import *
%cd munich_target_masks
import matplotlib.pyplot as plt
#import torch
#from torchvision.transforms import ToPILImage
import rasterio
import rioxarray
import folium
from rasterio import features
import shapely.geometry as sg
from tqdm import tqdm
from shapely.geometry import box
import torch.nn as nn

/pfs/data5/home/tu/tu_tu/tu_zxmav84/DS_Project/modules/models/segmentation
/pfs/data5/home/tu/tu_tu/tu_zxmav84/DS_Project/modules/models/segmentation/munich_target_masks


In [3]:
# Load config file and get orthophoto data path
import yaml
config_path = '/home/tu/tu_tu/tu_zxmav84/DS_Project/modules/config.yml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
orthophoto_dir = config['data']['orthophotos']
predictions_dir = config['data']['segmentation_output']
nutzungsdaten_dir = config['data']['nutzungsdaten']

### Read in Nutzungsdaten and Hausrumringe

In [4]:
nutzungsdaten_df = pickle.load(open(nutzungsdaten_dir + 'nutzungsdaten_relevant.pkl', 'rb'))
nutzungsdaten_df = nutzungsdaten_df[nutzungsdaten_df.Label.isin([4,6])]
nutzungsdaten_df.head(3)

,geometry,geometry_4326,Label
51,"POLYGON ((690768.450 5352601.390, 690769.740 5...","POLYGON ((11.57231 48.29783, 11.57233 48.29784...",4.0
52,"POLYGON ((690399.200 5352626.850, 690402.450 5...","POLYGON ((11.56735 48.29817, 11.56739 48.29822...",4.0
53,"POLYGON ((686751.820 5350328.940, 686752.260 5...","POLYGON ((11.51720 48.27860, 11.51721 48.27862...",4.0


In [5]:
hausumringe_df = pickle.load(open(config['data']['building_boxes'] + '/processed_building_boxes/building_boxes.pkl', 'rb'))
hausumringe_df['Label'] = 2
hausumringe_df

,ags,geometry,geometry_4326,Label
1270,9178124,"POLYGON ((700491.420 5360102.610, 700492.050 5...","POLYGON ((11.70683 48.36223, 11.70684 48.36225...",2
1271,9178124,"POLYGON ((700503.390 5360106.260, 700504.010 5...","POLYGON ((11.70700 48.36226, 11.70701 48.36228...",2
1272,9178124,"POLYGON ((700191.560 5360035.000, 700191.780 5...","POLYGON ((11.70276 48.36172, 11.70276 48.36174...",2
1273,9178124,"POLYGON ((700179.540 5359970.330, 700180.500 5...","POLYGON ((11.70256 48.36114, 11.70258 48.36117...",2
1274,9178124,"POLYGON ((700068.560 5360035.430, 700069.460 5...","POLYGON ((11.70110 48.36176, 11.70111 48.36180...",2
...,...,...,...,...
2658854,9175132,"POLYGON ((706983.580 5332582.230, 706985.770 5...","POLYGON ((11.78093 48.11283, 11.78096 48.11284...",2
2658855,9175132,"POLYGON ((707177.490 5332557.990, 707181.260 5...","POLYGON ((11.78352 48.11255, 11.78357 48.11256...",2
2658856,9175132,"POLYGON ((707148.520 5332599.170, 707151.260 5...","POLYGON ((11.78315 48.11293, 11.78319 48.11294...",2
2658857,9175132,"POLYGON ((707145.860 5332622.270, 707149.100 5...","POLYGON ((11.78313 48.11314, 11.78317 48.11315...",2


In [7]:
def create_target_mask(orig_tif_path, nutzungsdaten_df, hausumringe_df,  out_name, reproject=False):

    ############## Build new transform based on the size of the predictions and the geographic extend of the original image ###########
    # Read in original tif
    orig = rasterio.open(orig_tif_path)
    orig_profile = orig.profile
    orig_bounds = orig.bounds

    new_profile = orig.profile
    new_profile.update(count = 1)


    ################## NUTZUNGSDATEN ###########################
    # Subset nutzungsdaten df to relevant polygons and crop them to extend of the original tif
    bbox = box(*orig.bounds)
    # Subset to all polygons that intersect boudning box
    subset_df = nutzungsdaten_df[nutzungsdaten_df.geometry.intersects(bbox)].copy()
    if len(subset_df) == 0:
        nd_mask = np.zeros((625,625))
    else:
        # Clip the polygons to the bounding box extent
        subset_df['geometry'] = subset_df.geometry.intersection(bbox)

        # Save to tif
        with rasterio.open('temp.tif', 'w+', **new_profile) as out:
            out_arr = out.read(1)

            # this is where we create a generator of geom, value pairs to use in rasterizing
            shapes = ((geom,value) for geom, value in zip(subset_df.geometry, subset_df.Label))

            burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
            out.write_band(1, burned)

        # Read in tif again to use its values
        out = rasterio.open('temp.tif')
        nd_mask = out.read(1)

        # delete temporary tif file from disk
        os.remove('temp.tif')

    ##################### HAUSUMRINGE ######################################
    # Subset nutzungsdaten df to relevant polygons and crop them to extend of the original tif
    bbox = box(*orig.bounds)
    # Subset to all polygons that intersect boudning box
    subset_df = hausumringe_df[hausumringe_df.geometry.intersects(bbox)].copy()
    if len(subset_df) == 0:
        hu_mask = np.zeros((625,625))
    else:
        # Clip the polygons to the bounding box extent
        subset_df['geometry'] = subset_df.geometry.intersection(bbox)

        # Save to tif
        with rasterio.open('temp.tif', 'w+', **new_profile) as out:
            out_arr = out.read(1)

            # this is where we create a generator of geom, value pairs to use in rasterizing
            shapes = ((geom,value) for geom, value in zip(subset_df.geometry, subset_df.Label))

            burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
            out.write_band(1, burned)

        # Read in tif again to use its values
        out = rasterio.open('temp.tif')
        hu_mask = out.read(1)

        # delete temporary tif file from disk
        os.remove('temp.tif')
    
    final_mask = nd_mask
    final_mask[hu_mask == 2] = 2

    # Write final prediction to tif
    with rasterio.open(out_name, 'w', **new_profile) as dst:
        dst.write(final_mask, 1)

    if reproject:
        # Read in again, reproject and save again
        dst = rioxarray.open_rasterio(out_name)
        dst = dst.rio.reproject('EPSG:4326')
        dst.rio.to_raster(out_name)


In [8]:
orig_tif_path = '/pfs/work7/workspace/scratch/tu_zxmav84-ds_project/data/orthophotos/patched/32698_5332_patch_3_3.tif'

create_target_mask(orig_tif_path, nutzungsdaten_df,hausumringe_df, 'test.tif', reproject=False)

### Check combined target mask

In [9]:
src = rasterio.open('test.tif')
transform = src.transform
data = src.read()

class_labels = {0:'ignore', 1:'impervious', 2:'building', 3:'low vegetation', 4:'water', 5:'trees', 6:'road', 7:'train', 255:'ignore'}

# initialize empty GeoDataFrame to store all polygons
gdf = gpd.GeoDataFrame(columns = ['label', 'geometry'])
for class_index in np.unique(data):
    mask = data == class_index
    # Create a generator of polygon geometries
    shapes = features.shapes(mask.astype(rasterio.uint8), transform=transform)
    # Convert the geometries to Shapely polygons
    polygons = [sg.shape(shape) for shape, value in shapes if value == 1] # if condition necessary to get only positive mask polygons, not negative

    multipolygon = sg.MultiPolygon(polygons)
    data_to_append = {'label':class_labels[class_index], 'geometry':multipolygon}
    gdf = gpd.GeoDataFrame(pd.concat([gdf, pd.DataFrame([data_to_append])]))

gdf.crs = 'EPSG:25832'
gdf['geometry_4326'] = gdf.geometry.to_crs('EPSG:4326')


In [12]:
#gdf.geometry = gdf.geometry.to_crs('EPSG:4326')
styles = {
    'ignore':{"fillColor": "#000000", 'color':'#000000', 'weight':0, 'fillOpacity':0.5}, 
    'impervious':{"fillColor": "#ffffff", 'color':'#ffffff', 'weight':0, 'fillOpacity':0.5}, 
    'building':{"fillColor": "#ff00ff", 'color':'#ff00ff', 'weight':0, 'fillOpacity':0.5}, 
    'low vegetation':{"fillColor": "#00ff00", '#00ff00':'pink', 'weight':0, 'fillOpacity':0.5}, 
    'water':{"fillColor": "#0000ff", 'color':'#0000ff', 'weight':0, 'fillOpacity':0.5}, 
    'trees':{"fillColor": "#008200", 'color':'#008200', 'weight':0, 'fillOpacity':0.5}, 
    'road':{"fillColor": "#ff0000", 'color':'#ff0000', 'weight':0, 'fillOpacity':0.5}, 
    'train':{"fillColor": "#505050", 'color':'#505050', 'weight':0, 'fillOpacity':0.5}, 
}
m = folium.Map(location = [gdf.iloc[0,:].geometry_4326.centroid.y, gdf.iloc[0,:].geometry_4326.centroid.x], zoom_start = 16)
# Add ESRI Satellite Layer
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True).add_to(m)
# Add Semantic Segmentation Layer
for label in gdf.label.unique():
    folium.GeoJson(gdf[gdf.label == label].geometry_4326, style_function=lambda x, label=label: styles[label], name=label).add_to(m)
folium.LayerControl().add_to(m)
#m